# Model Pruning Experiments

This notebook demonstrates pruning experiments on:
1. Vision Transformer (ViT)
2. Deep Compression AutoEncoder (DC-AE)

In [1]:
# Clone the repository
!git clone https://github.com/tahahah/model-pruning-experiments.git
%cd model-pruning-experiments

# Install dependencies
!pip install -r requirements.txt

Cloning into 'model-pruning-experiments'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 124 (delta 59), reused 86 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (124/124), 464.35 KiB | 8.76 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/model-pruning-experiments
  Cloning https://github.com/mit-han-lab/efficientvit.git to /tmp/pip-req-build-f9tzhr1q
  Running command git clone --filter=blob:none --quiet https://github.com/mit-han-lab/efficientvit.git /tmp/pip-req-build-f9tzhr1q
  Resolved https://github.com/mit-han-lab/efficientvit.git to commit b94ff779828eea399c78f626b574da2d50ef2e49
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/alibaba/TinyNeuralNetwork.git to /tmp/pip-install-tpk6ff6c/tinyneuralnetwork_400aa4ee2dc845208465fd249e1c1df3
 

## 1. Vision Transformer (ViT) Pruning

In [ ]:
from vit_pruning_analysis import *

# Run ViT pruning analysis
main()

## 2. DC-AE Pruning

In [ ]:
!git stash && git pull

Saved working directory and index state WIP on master: 74f3dfa fix: incorrect import utils
Already up to date.


In [ ]:
!python vae_pruning_analysis.py

Loading DC-AE model...

Analyzing original model...

Model Sparsity Analysis:
Total Parameters: 676,475,584
Non-zero Parameters: 676,475,584 (100.0%)
Zero Parameters: 0 (0.0%)

Top 10 Most Sparse Layers:
                                                                      Layer      Size   NonZero  Sparsity(%)
147                          decoder.project_out.op_list.2.conv.conv.weight     27648     27648          0.0
0                                       encoder.project_in.conv.conv.weight      1728      1728          0.0
1                         encoder.stages.1.op_list.0.main.conv1.conv.weight    589824    589824          0.0
2                         encoder.stages.1.op_list.0.main.conv2.conv.weight    589824    589824          0.0
132          decoder.stages.5.op_list.2.context_module.main.qkv.conv.weight   3145728   3145728          0.0
133         decoder.stages.5.op_list.2.context_module.main.proj.conv.weight   1048576   1048576          0.0
134  decoder.stages.5.op_list.2.l

## 3. Finetuning DC AE

In [3]:
!pip install datasets
!wandb login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: htt

In [38]:
!git reset --hard && git pull


HEAD is now at 62438f0 fix: batch size 1 to avoid vram issues
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 588 bytes | 294.00 KiB/s, done.
From https://github.com/tahahah/model-pruning-experiments
   62438f0..ede3378  master     -> origin/master
Updating 62438f0..ede3378
Fast-forward
 dcaecore/trainer.py | 17 ++++++++++++++---
 1 file changed, 14 insertions(+), 3 deletions(-)


In [20]:
import os

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["LOCAL_RANK"] = "0"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

In [40]:
!python dcaecore/train_dc_ae.py --config dcaecore/config.yaml --pretrained "mit-han-lab/dc-ae-f32c32-in-1.0" --output_dir dcaecore/outputs --gpu 0

Resolving data files: 100% 339/339 [00:00<00:00, 345.17it/s]
Resolving data files: 100% 339/339 [00:00<00:00, 463149.53it/s]
len(train_dataset)=1000
len(val_dataset)=1000
len(test_dataset)=1000
2025-02-13 09:00:25,728 - dcae_training - INFO - Set random seed to 42
wandb: Currently logged in as: tahaa to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /content/model-pruning-experiments/wandb/run-20250213_090026-aibsj2dv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run worldly-violet-11
wandb: ⭐️ View project at https://wandb.ai/tahaa/dcae-finetuning
wandb: 🚀 View run at https://wandb.ai/tahaa/dcae-finetuning/runs/aibsj2dv
2025-02-13 09:00:27,257 - dcae_training - INFO - Initialized Weights & Biases logging
2025-02-13 09:00:27,258 - dcae_training - INFO - Create

## View Results

The results are saved in the `output` directory:
- Pruning analysis reports
- Reconstructed images
- Model statistics